# YOLOv8 Fruit Detection Training on Colab

This notebook trains YOLOv8 on the augmented fruit dataset.

**Before running:**
1. Upload your `yolo_dataset.zip` to Google Drive
2. Runtime > Change runtime type > GPU 

## Install YOLOv8

In [ ]:
!pip install ultralytics -q

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Extract Dataset

In [ ]:
!unzip -q /content/drive/MyDrive/yolo_dataset.zip -d /content/

# Verify dataset structure
!ls /content/yolo_dataset/

## Update Dataset YAML Path

In [ ]:
import yaml

# Read existing config
with open('/content/yolo_dataset/dataset.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update path to Colab location
config['path'] = '/content/yolo_dataset'

# Write updated config
with open('/content/yolo_dataset/dataset.yaml', 'w') as f:
    yaml.dump(config, f, sort_keys=False)

print(f"Dataset configured with {config['nc']} classes")
print(f"Train images: {config['train']}")
print(f"Val images: {config['val']}")

## GPU Validation

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Add GPU to runtime")

## Train YOLOv8


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# Train
results = model.train(
    data='/content/yolo_dataset/dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=128,
    name='fruit_detector',
    project='/content/drive/MyDrive/yolo_runs',
    patience=10,
    save=True,
    plots=True,
    device=0,
    workers=2,
    verbose=True
)

print("\nTraining completed!")
print(f"Best model saved to: /content/yolo_runs/fruit_detector/weights/best.pt")

## Test Model on Sample Images

In [ ]:
model = YOLO('/content/yolo_runs/fruit_detector/weights/best.pt')

# Run inference on validation images
results = model.predict(
    source='/content/yolo_dataset/images/val',
    save=True,
    conf=0.5,
    max_det=1,
    project='/content/predictions',
    name='val_predictions'
)

print("Predictions saved to: /content/predictions/val_predictions/")

## Model Metrics and Confusion Matrix

In [ ]:
from ultralytics import YOLO

# Load the best model (from training or from Google Drive)
model = YOLO('/content/drive/MyDrive/yolo_runs/fruit_detector5weights/best.pt')

# Run validation to generate metrics and plots
print("Running validation...")
metrics = model.val(
    data='/content/yolo_dataset_room/dataset.yaml',
    split='val',
    save_json=True,
    plots=True,
    device=0
)

print(f"\n✓ Validation complete!")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

# View results
from IPython.display import Image, display
import glob

val_run = sorted(glob.glob('/content/runs/detect/val*'))[-1]
print(f"\nResults saved to: {val_run}\n")

print("Confusion Matrix:")
display(Image(filename=f'{val_run}/confusion_matrix.png', width=800))